# PU Characterization on nu-gun samples
#TOC

   * [TC level plots](#Trigger-Cell-level-plots)
   * [3D cluster level plots](#2D-cluster-level-plots)
   * [3D cluster level plots](#3D-cluster-plots)

In [75]:
import sys
sys.path.insert(0, '..')
import ROOT
import root_numpy as rnp
import l1THistos as histos
import math
import uuid

import pandas as pd

ROOT.enableJSVis()


    

In [39]:
# %load ../drawingTools
import ROOT
import math
import uuid

# some useful globals, mainly to deal with ROOT idiosyncrasies
c_idx = 0
p_idx = 0
colors = range(1, 6)
stuff = []
f_idx = 0

ROOT.gStyle.SetPadBottomMargin(0.13)
ROOT.gStyle.SetPadLeftMargin(0.13)
ROOT.gStyle.SetPadRightMargin(0.30)

ROOT.gStyle.SetCanvasBorderMode(0)
ROOT.gStyle.SetCanvasColor(0)
ROOT.gStyle.SetCanvasDefH(600)
ROOT.gStyle.SetCanvasDefW(800)


# define some utility functions
def newCanvas(name=None, title=None, xdiv=0, ydiv=0, form=4):
    global c_idx
    if name is None:
        name = 'c_{}'.format(uuid.uuid4().hex[:6])
        c_idx += 1
    if title is None:
        title = name
    # print name, title
    canvas = ROOT.TCanvas(name, title)
    if(xdiv*ydiv != 0):
        canvas.Divide(xdiv, ydiv)
    return canvas


def draw(plot, options='', text=None):
    c = newCanvas()
    c.cd()
    plot.Draw(options)
    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw('same')

    c.Draw()

    return


def getLegend(x1=0.7, y1=0.71, x2=0.95, y2=0.85):
    global stuff
    legend = ROOT.TLegend(x1, y1, x2, y2)
    stuff.append(legend)
    legend.SetFillColor(0)
    legend.SetFillStyle(0)
    legend.SetBorderSize(0)
    legend.SetTextSize(0.05)
    return legend


def drawAndProfileX(plot2d, miny=None, maxy=None, do_profile=True, options='', text=None):
    global p_idx
    if miny and maxy:
        plot2d.GetYaxis().SetRangeUser(miny, maxy)
    c = newCanvas()
    c.SetGrid(1, 1)
    c.cd()
    plot2d.Draw(options)
    ROOT.gPad.SetGrid(1, 1)
    ROOT.gStyle.SetGridColor(15)

    if do_profile:
        profname = plot2d.GetName()+'_prof_'+str(p_idx)
        p_idx += 1
        firstbin = 1
        lastbin = -1
        prof = plot2d.ProfileX(profname, firstbin, lastbin, 's')
        prof.SetMarkerColor(2)
        prof.SetLineColor(2)
        prof.Draw('same')

    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw('same')

    c.Draw()


class Sample():
    def __init__(cls, name, label, version=None):
        cls.name = name
        cls.label = label
        if version:
            version = '_'+version
        else:
            version = ''
        cls.histo_filename = '../plots1/histos_{}{}.root'.format(cls.name, version)
        cls.histo_file = ROOT.TFile(cls.histo_filename)


def getText(text, ndc_x, ndc_y):
    global stuff
    rtext = ROOT.TLatex(ndc_x, ndc_y, text)
    stuff.append(rtext)
    rtext.SetNDC(True)
    # rtext.SetTextFont(40)
    rtext.SetTextSize(0.03)
    return rtext


def drawSame(histograms, labels, options='', norm=False, logy=False, min_y=None, max_y=None, text=None):
    global colors
    c = newCanvas(title=histograms[0].GetName())
    c.cd()
    leg = getLegend()

    max_value = max_y
    min_value = min_y
    if min_y is None:
        min_value = min([hist.GetBinContent(hist.GetMinimumBin()) for hist in histograms])
    if max_y is None:
        max_value = max([hist.GetBinContent(hist.GetMaximumBin()) for hist in histograms])*1.2

    for hidx, hist in enumerate(histograms):
        hist.SetLineColor(colors[hidx])
        hist.SetStats(False)
        if norm:
            hist.DrawNormalized('same'+','+options, 1.)
        else:
            if hidx:
                hist.Draw('same'+','+options)
            else:
                hist.Draw(options)
        leg.AddEntry(histograms[hidx], labels[hidx], 'l')

    histograms[0].GetYaxis().SetRangeUser(min_value, max_value)
    leg.Draw()
    c.Draw()
    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw("same")
    if logy:
        c.SetLogy()


def drawProfileX(histograms, labels, options=''):
    profiles = [hist.ProfileX() for hist in histograms]
    drawSame(profiles, labels, options)


def drawSeveral(histograms, labels, options='', do_profile=False, miny=None, maxy=None, text=None):
    ydiv = int(math.ceil(float(len(histograms))/2))
    for hidx in range(0, len(histograms)):
        newtext = labels[hidx]
        if text:
            newtext = '{}: {}'.format(labels[hidx], text)
        if do_profile:
            drawAndProfileX(histograms[hidx], miny=miny, maxy=maxy, options=options, do_profile=do_profile, text=newtext)
        else:
            draw(histograms[hidx], options=options, text=newtext)


def drawProfileRatio(prof1, prof2, ymin=None, ymax=None, text=None):
    hist1 = prof1.ProjectionX(uuid.uuid4().hex[:6])
    hist2 = prof2.ProjectionX(uuid.uuid4().hex[:6])
    hist1.Divide(hist2)
    draw(hist1)
    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw("same")

    if ymin is not None and ymax is not None:
        hist1.GetYaxis().SetRangeUser(ymin, ymax)
    ROOT.gPad.Update()
        
# mean+-nsigmas*RMS.
def drawGaussFit(histo, nsigmas, min, max):
    minfit = histo.GetMean() - nsigmas*histo.GetRMS()
    maxfit = histo.GetMean() + nsigmas*histo.GetRMS()
    drawGFit(histo, min, max, minfit, maxfit)


# Fit a histogram in the range (minfit, maxfit) with a gaussian and
# draw it in the range (min, max)
def drawGFit(histo, min, max, minfit, maxfit):
    # static int i = 0
    # i++
    # gPad->SetGrid(1,1);
    # gStyle->SetGridColor(15);
    histo.GetXaxis().SetRangeUser(min,max)
    global f_idx
    nameF1 = "g{}".format(f_idx)
    f_idx +=1
    g1 = ROOT.TF1(nameF1,"gaus",minfit,maxfit)
    g1.SetLineColor(2)
    g1.SetLineWidth(2)
    histo.Fit(g1,"R")



def drawGraphsSame(histograms, labels, options='', norm=False, logy=False, min_y=None, max_y=None, text=None):
    global colors
    c = newCanvas()
    c.cd()
    leg = getLegend()

    for hidx in range(0, len(histograms)):
        histograms[hidx].SetLineColor(colors[hidx])
        histograms[hidx].Draw('same'+','+options)
        leg.AddEntry(histograms[hidx], labels[hidx], 'l')

    leg.Draw()
    c.Draw()
    if logy:
        c.SetLogy()
    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw("same")


In [90]:
version = 'v32'

#      Particle('ele', PID.electron),
#                  Particle('ele_A', PID.electron, '1.4 < abseta < 1.7'),
#                  Particle('ele_B', PID.electron, '1.7 <= abseta <= 2.8'),
#                  Particle('ele_C', PID.electron, 'abseta > 2.8'),
#                  Particle('photon', PID.photon),
#                  Particle('photon_A', PID.photon, '1.4 < abseta < 1.7'),
#                  Particle('photon_B', PID.photon, '1.7 <= abseta <= 2.8'),
#                  Particle('photon_C', PID.photon, 'abseta > 2.8'),
                
particles = ''

sample_names = ['ele_flat2to100_PU0',
                'ele_flat2to100_PU200',
                'photonPt35_PU0',
                'photonPt35_PU200']


sample_ele_flat2to100_PU0 = Sample('ele_flat2to100_PU0', 'PU0', version)
sample_ele_flat2to100_PU200 = Sample('ele_flat2to100_PU200', 'PU200', version)

sample_gPt35_PU0 = Sample('photonPt35_PU0', 'Pt35 PU0', version)
sample_gPt35_PU200 = Sample('photonPt35_PU200', 'Pt35 PU200', version)

sample_hadronGun_PU0 = Sample('hadronGun_PU0', 'PU0', version)
sample_hadronGun_PU200 = Sample('hadronGun_PU200', 'PU200', version)


sample_nuGun_PU50 = Sample('nuGun_PU50', 'PU50', version)
sample_nuGun_PU100 = Sample('nuGun_PU100', 'PU100', version)
sample_nuGun_PU140 = Sample('nuGun_PU140', 'PU140', version)
sample_nuGun_PU200 = Sample('nuGun_PU200', 'PU200', version)



samples_ele = [sample_ele_flat2to100_PU0,sample_ele_flat2to100_PU200]
samples_photon = [sample_gPt35_PU0,sample_gPt35_PU200]
samples_hadrons = [sample_hadronGun_PU0,sample_hadronGun_PU200]
samples_nuguns = [sample_nuGun_PU50, sample_nuGun_PU100, sample_nuGun_PU140, sample_nuGun_PU200]


samples = []


Error in <TFile::TFile>: file ../plots1/histos_photonPt35_PU0_v32.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_photonPt35_PU200_v32.root does not exist


In [72]:

samples = samples_nuguns
pu_labels = [sample.label for sample in samples]


tpset_labels = {'DEF' : 'NNDR',
                'DEF_em': 'NNDR + EGID'}


particle_labels = {'ele': 'all #eta',
                   'eleA' : '1.4 < |#eta| < 1.7',
                   'eleB' : '1.7 < |#eta| < 1.8',
                   'eleC' : '|#eta| > 2.8',
                   'nomatch': 'nomatch',
                   }



In [19]:
particles = ['nomatch']
tp_sets = ['DEF']

histo_hGenDen_df = pd.DataFrame(index=tp_sets, columns=particles)
histo_hcl3d_df = pd.DataFrame(index=tp_sets, columns=particles)
histo_hcl2d_df = pd.DataFrame(index=tp_sets, columns=particles)
histo_htc_df = pd.DataFrame(index=tp_sets, columns=particles)
titles_df = pd.DataFrame(index=tp_sets, columns=particles)


for tp_set in tp_sets:
    for particle in particles:
        histo_hGenDen_df.loc[tp_set][particle] =  [histos.HistoSetEff('{}_{}'.format(tp_set, particle), sample.histo_file).h_den for sample in samples]
        histo_hcl3d_df.loc[tp_set][particle] =  [histos.HistoSetClusters('{}_{}'.format(tp_set, particle), sample.histo_file).hcl3d for sample in samples]
        histo_hcl2d_df.loc[tp_set][particle] =  [histos.HistoSetClusters('{}_{}'.format(tp_set, particle), sample.histo_file).hcl2d for sample in samples]
        histo_htc_df.loc[tp_set][particle] =  [histos.HistoSetClusters('{}_{}'.format(tp_set, particle), sample.histo_file).htc for sample in samples]
        histo_htc_df.loc[tp_set][particle] =  [histos.HistoSetClusters('{}_{}'.format(tp_set, particle), sample.histo_file).htc for sample in samples]

        titles_df.loc[tp_set, particle] = '{}, {}'.format(tpset_labels[tp_set], particle_labels[particle])

        

# NuGun samples with different PU

## Trigger Cell level plots

In [22]:
tp_set = 'DEF'
particle = 'nomatch'

htc_energies = [histo.h_energy for histo in histo_htc_df.loc[tp_set, particle]]
drawSame(htc_energies, pu_labels,'hist', logy=True)
drawSame(htc_energies, pu_labels,'hist', norm=True, logy=True)



In [23]:
htc_layerVenergy = [histo.h_layerVenergy for histo in histo_htc_df.loc[tp_set, particle]]
drawSeveral(htc_layerVenergy, pu_labels,'COLZ',do_profile=True)



In [31]:
htc_layerVenergy_proj_L3 = [histo.h_layerVenergy.ProjectionY(uuid.uuid4().hex[:6], 4, 4) for histo in histo_htc_df.loc[tp_set, particle]]
drawSame(htc_layerVenergy_proj_L3, pu_labels,'hist', logy=True, text='Layer 3')
drawSame(htc_layerVenergy_proj_L3, pu_labels,'hist', logy=True, text='Layer 3, morm=1', norm=True)

htc_layerVenergy_proj_L9 = [histo.h_layerVenergy.ProjectionY(uuid.uuid4().hex[:6], 10, 10) for histo in histo_htc_df.loc[tp_set, particle]]
drawSame(htc_layerVenergy_proj_L9, pu_labels,'hist', logy=True, text='Layer 9')
drawSame(htc_layerVenergy_proj_L9, pu_labels,'hist', logy=True, text='Layer 9, morm=1', norm=True)


htc_layerVenergy_proj_L15 = [histo.h_layerVenergy.ProjectionY(uuid.uuid4().hex[:6], 16, 16) for histo in histo_htc_df.loc[tp_set, particle]]
drawSame(htc_layerVenergy_proj_L15, pu_labels,'hist', logy=True, text='Layer 15')
drawSame(htc_layerVenergy_proj_L15, pu_labels,'hist', logy=True, text='Layer 15, morm=1', norm=True)


The plot below shows the average TC energy per layer # (integrated over all eta) for different PU.

In [40]:
htc_profLayerVenergy = [histo.h_layerVenergy.ProfileX(uuid.uuid4().hex[:6]) for histo in histo_htc_df.loc[tp_set, particle]]
drawSame(htc_profLayerVenergy, pu_labels)

line = ROOT.TLine(0,1,60,1)
line.SetLineColor(2)
line.SetLineStyle(2)

#PU200/PU100
drawProfileRatio(htc_profLayerVenergy[3], htc_profLayerVenergy[2], ymin=0.8, ymax=1.3, text='PU200/PU100')
line.Draw('same')

#PU100/PU500
drawProfileRatio(htc_profLayerVenergy[1], htc_profLayerVenergy[0], ymin=0.8, ymax=1.3, text='PU100/PU50')
line.Draw('same')


In [41]:
htc_layer = [histo.h_layer for histo in histo_htc_df.loc[tp_set, particle]]
drawSame(htc_layer, pu_labels)

In [43]:
# ration PU200 PU100
drawProfileRatio(htc_layer[3], htc_layer[1], ymin=0, ymax=4, text='PU200/PU100')
line2 = ROOT.TLine(0,2,60,2)
line2.SetLineColor(2)
line2.SetLineStyle(2)
line2.Draw('same')

drawProfileRatio(htc_layer[1], htc_layer[0], ymin=0, ymax=4, text='PU100/PU50')
line3 = ROOT.TLine(0,2,60,2)
line3.SetLineColor(2)
line3.SetLineStyle(2)
line3.Draw('same')

The # of TC per layer varies ~linearly with the PU, confirming that, apart from high-eta regions, the TC can "resolve" the PU.
The behavior vs eta is addressed in the following plots.


In [45]:


htcAll_energyPetaVphi = [histo.h_energyPetaVphi for histo in histo_htc_df.loc[tp_set, particle]]
drawSeveral(htcAll_energyPetaVphi, pu_labels,'COLZ',do_profile=False)


In [46]:
htcAll_energyVeta = [histo.h_energyVeta for histo in histo_htc_df.loc[tp_set, particle]]
drawSeveral(htcAll_energyVeta, pu_labels,'COLZ',do_profile=True)

In [47]:
htcAll_profEnergyVeta = [histo.h_energyVeta.ProfileX(uuid.uuid4().hex[:6]) for histo in histo_htc_df.loc[tp_set, particle]]
drawSame(htcAll_profEnergyVeta, pu_labels)

In [49]:
for hist in htcAll_profEnergyVeta:
    hist.GetYaxis().SetRangeUser(0.08,0.5)
drawSame(htcAll_profEnergyVeta, pu_labels)

line = ROOT.TLine(-4,1,+4,1)
line.SetLineColor(2)
line.SetLineStyle(2)
#PU100/PU50
drawProfileRatio(htcAll_profEnergyVeta[1], htcAll_profEnergyVeta[0], ymin=0.8, ymax=1.2, text='PU100/PU50')
line.Draw('same')


In [50]:
#PU200/PU100
drawProfileRatio(htcAll_profEnergyVeta[3], htcAll_profEnergyVeta[2], ymin=0.8, ymax=1.2, text='PU200/PU100')
line.Draw('same')

In [52]:
htcAll_profEnergyVetaL1t5 = [histo.h_energyVetaL1t5.ProfileX(uuid.uuid4().hex[:6]) for histo in histo_htc_df.loc[tp_set, particle]]
htcAll_profEnergyVetaL6t10 = [histo.h_energyVetaL6t10.ProfileX(uuid.uuid4().hex[:6]) for histo in histo_htc_df.loc[tp_set, particle]]
htcAll_profEnergyVetaL11t20 = [histo.h_energyVetaL11t20.ProfileX(uuid.uuid4().hex[:6]) for histo in histo_htc_df.loc[tp_set, particle]]
htcAll_profEnergyVetaL21t60 = [histo.h_energyVetaL21t60.ProfileX(uuid.uuid4().hex[:6]) for histo in histo_htc_df.loc[tp_set, particle]]

#PU200/PU100
drawProfileRatio(htcAll_profEnergyVetaL1t5[3], htcAll_profEnergyVetaL1t5[2], ymin=0.8, ymax=1.2, text='PU200/PU100, layers 1 to 5')
line.Draw('same')
drawProfileRatio(htcAll_profEnergyVetaL6t10[3], htcAll_profEnergyVetaL6t10[2], ymin=0.8, ymax=1.2, text='PU200/PU100, layers 6 to 10')
line.Draw('same')
drawProfileRatio(htcAll_profEnergyVetaL11t20[3], htcAll_profEnergyVetaL11t20[2], ymin=0.8, ymax=1.2, text='PU200/PU100, layers 11 to 20')
line.Draw('same')
drawProfileRatio(htcAll_profEnergyVetaL21t60[3], htcAll_profEnergyVetaL21t60[2], ymin=0.8, ymax=1.2, text='PU200/PU100, layers 21 to 60')
line.Draw('same')

## 2D cluster level plots

In [53]:
histo_hcl2d_df

,nomatch
DEF,"[<ClusterHistos h_cl2d_DEF_nomatch>, <ClusterH..."


In [55]:

hclAll_energies = [histo.h_energy for histo in histo_hcl2d_df.loc[tp_set, particle]]

drawSame(hclAll_energies, pu_labels,'hist', logy=True)
drawSame(hclAll_energies, pu_labels,'hist',norm=True, logy=True, text='normalized')



In [57]:
hclAll_layers = [histo.h_layer for histo in histo_hcl2d_df.loc[tp_set, particle]]
drawSame(hclAll_layers, pu_labels,'hist', logy=True)
drawSame(hclAll_layers, pu_labels,'hist', norm=True, logy=True)


In [58]:
hclAll_ncells = [histo.h_ncells for histo in histo_hcl2d_df.loc[tp_set, particle]]
drawSame(hclAll_ncells, pu_labels,'hist', logy=True)


In [60]:
hclAll_layerVenergy = [histo.h_layerVenergy for histo in histo_hcl2d_df.loc[tp_set, particle]]
drawSeveral(hclAll_layerVenergy, pu_labels,'COLZ',do_profile=True)

In [61]:
hclAll_profLayerVenergy = [histo.h_layerVenergy.ProfileX(uuid.uuid4().hex[:6]) for histo in histo_hcl2d_df.loc[tp_set, particle]]
drawSame(hclAll_profLayerVenergy, pu_labels)


In [63]:
hclAll_layerVncells = [histo.h_layerVncells for histo in histo_hcl2d_df.loc[tp_set, particle]]
drawSeveral(hclAll_layerVncells, pu_labels,'COLZ',do_profile=True)

In [64]:
hclAll_profLayerVncells = [histo.h_layerVncells.ProfileX(uuid.uuid4().hex[:6]) for histo in histo_hcl2d_df.loc[tp_set, particle]]
drawSame(hclAll_profLayerVncells, pu_labels)

In [69]:
#We plot the average energy per TC in a cluster

#PU 50 nugun
drawProfileRatio(hclAll_profLayerVenergy[0], hclAll_profLayerVncells[0], ymin=None, ymax=None, text='PU50')


In [70]:
# we do the same for the PU200 nu gun
drawProfileRatio(hclAll_profLayerVenergy[3], hclAll_profLayerVncells[3], ymin=None, ymax=None, text='PU200')


## 3D cluster plots

In [73]:
### 3D clusters???
histo_hcl3d_df

,nomatch
DEF,"[<Cluster3DHistos h_cl3d_DEF_nomatch>, <Cluste..."


In [79]:

h3DclAll_energies = [histo.h_energy for histo in histo_hcl3d_df.loc[tp_set, particle]]
drawSame(h3DclAll_energies, pu_labels,'hist', logy=True)
drawSame(h3DclAll_energies, pu_labels,'hist', logy=True, norm=True)


In [80]:

h3DclAll_eta = [histo.h_eta for histo in histo_hcl3d_df.loc[tp_set, particle]]
drawSame(h3DclAll_eta, pu_labels,'hist', logy=True)
drawSame(h3DclAll_eta, pu_labels,'hist', logy=True, norm=True)


In [85]:

h3DclAll_ncluVpt = [histo.h_ncluVpt for histo in histo_hcl3d_df.loc[tp_set, particle]]
drawSeveral(h3DclAll_ncluVpt, pu_labels,'COLZ')


In [83]:
h3DclAll_nclu = [histo.h_nclu for histo in histo_hcl3d_df.loc[tp_set, particle]]
drawSame(h3DclAll_nclu, pu_labels,'hist', logy=True)
drawSame(h3DclAll_nclu, pu_labels,'hist', logy=True, norm=True)


In [86]:
hTT_all = [histos.TriggerTowerHistos('h_TT_all', sample.histo_file) for sample in samples]


In [88]:
hTTAll_energy = [histo.h_energy for histo in hTT_all]
drawSame(hTTAll_energy, pu_labels,'hist', logy=True)


In [89]:
hTTAll_HoE = [histo.h_HoE for histo in hTT_all]
drawSame(hTTAll_HoE, pu_labels,'hist', logy=True)
